In [1]:
from pymongo import MongoClient
import pandas as pd
from pykrx import stock
from datetime import datetime,timedelta,date
from googletrans import Translator
import numpy as np
import requests
import re
import bs4
from transformers import pipeline
import time
from pykrx import stock
#import FinanceDataReader as fdr
import requests
from bs4 import BeautifulSoup


qa = pipeline("question-answering")
zs = pipeline('zero-shot-classification')


import warnings
warnings.filterwarnings('ignore')

def craw(comp):
    if comp == 'kakao':
        code = '035720'  # kakao
    else:
        code = '035420'  # naver

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.151 Whale/3.14.134.62 Safari/537.36'}

    date_list = []
    title_list = []
    info_list = []
    url_list = []

    sub_date_list = []
    sub_title_list = []
    sub_info_list = []
    sub_url_list = []

    for page in range(1, 3):
        url = f'https://finance.naver.com/item/news_news.naver?code={code}&page={page}&sm=title_entity_id.basic&clusterId='
        res = requests.get(url, headers=headers)
        bs = bs4.BeautifulSoup(res.text, 'html.parser')

        # 뉴스 전체 크롤링
        art_cnt = len(bs.find_all('td', {'class': 'title'}))
        for i in range(art_cnt):
            date_list.append(bs.find_all('td', {'class': 'date'})[i].text.split()[0].replace('.', '-'))
            title_list.append(bs.find_all('td', {'class': 'title'})[i].text.replace('\n', ''))
            info_list.append(bs.find_all('td', {'class': 'info'})[i].text)
            url_list.append('https://finance.naver.com' + bs.find_all('td', {'class': 'title'})[i].find('a')['href'])

        # 관련 뉴스만 크롤링
        chk = 1
        while True:
            try:
                chk_len = len(bs.find_all('table', {'class': 'type5'})[chk].find_all('td', {'class': 'title'}))
                for i in range(chk_len):
                    sub_date_list.append(
                        bs.find_all('table', {'class': 'type5'})[chk].find_all('td', {'class': 'date'})[i].text.split()[
                            0].replace('.', '-'))
                    sub_title_list.append(
                        bs.find_all('table', {'class': 'type5'})[chk].find_all('td', {'class': 'title'})[
                            i].text.replace('\n', ''))
                    sub_info_list.append(
                        bs.find_all('table', {'class': 'type5'})[chk].find_all('td', {'class': 'info'})[i].text)
                    sub_url_list.append('https://finance.naver.com' +
                                        bs.find_all('table', {'class': 'type5'})[chk].find_all('td',
                                                                                               {'class': 'title'})[
                                            i].find('a')['href'])
                chk += 1
            except:
                break
    df = pd.DataFrame()
    df['날짜'] = date_list
    df['제목'] = title_list
    df['정보제공'] = info_list
    df['링크'] = url_list

    df2 = pd.DataFrame()  # 관련기사
    df2['날짜'] = sub_date_list
    df2['제목'] = sub_title_list
    df2['정보제공'] = sub_info_list
    df2['링크'] = sub_url_list

    result = pd.concat([df,df2])
    result = result.drop_duplicates('제목',keep=False)
    today = datetime.strftime(datetime.now().date(), '%Y-%m-%d')
    # today = "2022-06-04"
    result = result[result['날짜']==today]

    # sub
    url_list = result['링크'].to_list()
    #print(url_list)
    art_list = []
    for i in range(len(url_list)):
        url = url_list[i]
        res = requests.get(url, headers = headers)
        bs = bs4.BeautifulSoup(res.text, 'html.parser')
        art_list.append(bs.find_all('div',{'class':'scr01'})[0].text)
        print(i)

    tst = []
    for art in art_list:
        tp = art.replace('\n|\t|\xa0|\u200b',' ') # \n,\t,\xa0,\u200b 삭제
        tp = tp.replace("'|‘|’",'"') # 따옴표 모두 큰따옴표로 통일
        tp = tp.replace('↑','상승').replace('↓','하락') # ↑↓ 치환
        tp = tp.replace('\([^\(\)]*?\)|\[[^\[\]]*?\]|<[^<>]*?>',' ').replace('\([^\(\)]*?\)|\[[^\[\]]*?\]|<[^<>]*?>',' ') # (),[],<> 제거
        tp = tp.replace('·+',',') # ·을 ,로 치환
        tp = tp.replace('[가-힣]{2,3} ?기자',' ') # 홍길동 기자 삭제
        tp = tp.replace('[a-z0-9]+@',' ') # 기자 태그 삭제
        tp = tp.replace('사진=|사진 ?제공=',' ') # 사진 태그 삭제
        tp = tp.replace('[^ 0-9ㄱ-ㅎ가-힣a-zA-Z"%&,.~-]', ' ')  # 특수문자 삭제
        tp = tp.replace('\.+', '.')  # . 2개 이상이면 1개만 남김
        tp = tp.replace(' +', ' ')  # 공백 2개 이상이면 1개만 남김
        tp = tp.replace("\'","")
        tp = tp.strip()  # 좌우 공백 제거
        tst.append(tp)

    translator = Translator()

    cnt = 0
    trs_art = []
    for i in tst:
        cnt += 1
        try:
            entext = translator.translate(i, src = 'ko', dest = 'en').text
            time.sleep(1)
            trs_art.append(entext)
        except:
            trs_art.append('')
            print(f'{cnt}번째는 번역 실패')

    return result,trs_art,today

def daily_score(result,trs_art,question,label):
    score = []

    for sequence in trs_art:
        if sequence == '':
            pass
        else:
            sequence = qa(question=question,context=sequence)
            temp = zs(sequence.get('answer'),label)
            temp_score = abs(temp['scores'][0]-temp['scores'][1])
            #cnt += 1

            if temp['labels'][0] == label[0]:
                score.append(-temp_score)
                #print(f'{cnt}/{len(trs_art)}' + f': append -{temp_score}')
            else:
                score.append(temp_score)
                #print(f'{cnt}/{len(trs_art)}' + f': append {temp_score}')

    return score

def daily_mongo_update(comp,db_location):
    #comp = 'kakao'
    result,trs_art,today = craw(comp)
    print('score1')
    question1 = f'Has the price of {comp} stock increased from the previous day?'
    label1 = ['price positive', 'price negative']
    tp1 = daily_score(result,trs_art, question1, label1)

    print('score2')
    question2 = f'Will the situation for the {comp} company improve in the future?'
    label2 = ['situation POSITIVE','situation NEGATIVE']
    tp2 = daily_score(result,trs_art,question2,label2)

    print('score3')
    question3 = f'Can {comp} Grow in the Future?'
    label3 = ['grow POSITIVE', 'grow NEGATIVE']
    tp3 = daily_score(result,trs_art, question3, label3)

    s_1 = pd.Series(tp1)
    s_2 = pd.Series(tp2)
    s_3 = pd.Series(tp3)
    df = pd.concat([s_1,s_2,s_3],axis=1)
    df.columns = ['score1','score2','score3']
    df["날짜"] = today
    df = df.groupby('날짜').mean().reset_index()
    df = df.set_index("날짜")
    return df

def get_today_data(comp):
    if comp == 'kakao':
        code = '035720'  # kakao
    else:
        code = '035420'  # naver
    # 오늘날짜
    a = datetime.now()
    year = f"{a.year}"
    month = f"{a.month}"
    if len(month) < 2:
        month = "0" + month
    day = f"{a.day}"
    if len(day) < 2:
        day = "0" + day
    today = year + month + day
    today_dash = year + "-" + month + "-" + day
    # today = "20220604"
    # today_dash = "2022-06-04"

    # 어제날짜
    b = date.today() - timedelta(7)
    year = f"{b.year}"
    month = f"{b.month}"
    if len(month) < 2:
        month = "0" + month
    day = f"{b.day}"
    if len(day) < 2:
        day = "0" + day
    yesterday = year + month + day

    # 나스닥 받아오기, df_ixic["Close"] 사용
    try:
        df_ixic = fdr.DataReader('IXIC', today_dash)
        df_ixic = df_ixic[["Close"]]
        df_ixic.columns = [["나스닥"]]
        df_ixic = df_ixic.set_index(pd.Index([today_dash]))
    except:
        df = pd.DataFrame([None],columns=["나스닥"])
        df_ixic = df.set_index(pd.Index([today_dash]))

    # pykrx로 받아오기
    # 코스피["종가"], 코스닥["종가"]
    try:
        df_kospi = stock.get_index_fundamental(today, today, "1001")
        df_kospi = df_kospi[["종가"]]
        df_kospi.columns = [["코스피"]]
        df_kospi = df_kospi.set_index(pd.Index([today_dash]))
        df_kosdaq = stock.get_index_fundamental(today, today, "2001")
        df_kosdaq = df_kosdaq[["종가"]]
        df_kosdaq.columns = [["코스닥"]]
        df_kosdaq = df_kosdaq.set_index(pd.Index([today_dash]))
    except:
        df = pd.DataFrame([None],columns=["코스피"])
        df_kospi = df.set_index(pd.Index([today_dash]))
        df = pd.DataFrame([None],columns=["코스닥"])
        df_kosdaq = df.set_index(pd.Index([today_dash]))

    # "기관합계매수량(원)","기타법인매수량(원)","개인매수량(원)","외국인합계매수량(원)"
    try:
        df_purchase = stock.get_market_trading_value_by_date(today, today, code)
        df_purchase.columns = [["기관합계매수량(원)","기타법인매수량(원)","개인매수량(원)","외국인합계매수량(원)","전체"]]
        df_purchase = df_purchase[["기관합계매수량(원)","기타법인매수량(원)","개인매수량(원)","외국인합계매수량(원)"]]
        df_purchase = df_purchase.set_index(pd.Index([today_dash]))
    except:
        df = pd.DataFrame([[None,None,None,None]],columns=["기관합계매수량(원)","기타법인매수량(원)","개인매수량(원)","외국인합계매수량(원)"])
        df_purchase = df.set_index(pd.Index([today_dash]))

    # 거래량,전일비
    try:
        df = stock.get_market_ohlcv(today, today, code,adjusted=False)
        df_volume = df[["거래량"]]
        df_volume = df_volume.set_index(pd.Index([today_dash]))
        df_end = df[["종가"]]
        df_end = df_end.set_index(pd.Index([today_dash]))
        df_updown = df[["등락률"]]
        df_updown = df_updown.set_index(pd.Index([today_dash]))
    except:
        df = pd.DataFrame([[None,None,None]], columns=["거래량","종가","등락률"])
        df_volume = df[["거래량"]]
        df_volume = df_volume.set_index(pd.Index([today_dash]))
        df_end = df[["종가"]]
        df_end = df_end.set_index(pd.Index([today_dash]))
        df_updown = df[["등락률"]]
        df_updown = df_updown.set_index(pd.Index([today_dash]))

    # 전일비
    try:
        df = stock.get_market_ohlcv(yesterday,today, code)
        df_expense = df["종가"]
        expense = []
        for i in range(len(df_expense)-1,len(df_expense)):
            expense.append(df_expense[i] - df_expense[i-1])
        df_expense = pd.DataFrame(expense,columns=[["전일비"]])
        df_expense = df_expense.set_index(pd.Index([today_dash]))
    except:
        df = df = pd.DataFrame([None], columns=["전일비"])
        df_expense = df[["전일비"]]
        df_expense = df_expense.set_index(pd.Index([today_dash]))

    df_all = pd.concat([df_ixic,df_kospi,df_kosdaq,df_purchase,df_end,df_updown,df_volume,df_expense],axis=1)
    df_all.columns = [["나스닥","코스피","코스닥",
                       "기관합계매수량(원)","기타법인매수량(원)","개인매수량(원)","외국인합계매수량(원)",
                       "종가","등락률","거래량","전일비"]]

    return df_all

def get_vix(url='https://kr.investing.com/indices/kospi-volatility-historical-data'):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}
    res = requests.get(url, headers = headers)
    bs = BeautifulSoup(res.text, 'html.parser')

    # VIX 지수 테이블
    table = bs.find('table', {'id': 'curr_table'})
    
    # 오늘 날짜와 동일한지 확인
    date_ = table.find('td', {'class': 'first left bold noWrap'}).text
    date_ = re.sub(r'[가-힣] ', '-', date_)[:10]
    
    if date_ == str(date.today()):
        # vix_ = table.find('td',{'class':'greenFont'}).text
        # today_vix = pd.DataFrame([vix_],columns=["vix"])
        # today_vix.set_index(pd.Index([date_]),inplace=True)
        today_vix = table.find('td',{'class':'greenFont'}).text
    else: # 오늘 날짜와 다를 경우에는 None 값을 넣음 
        today_vix = None
    
    return today_vix

# 이전 데이터를 db에서 불러오는 함수로 만들면됨. 마지막 한줄만 불러와도됨.
# 다만 형태는 밑에 적혀있는대로 되어야함.
# 나스닥,코스피,코스닥,기관합계매수량,기타법인매수량,개인매수량,외국인합계매수량,종가,등락률,거래량,전일비,score1,score2,score3,변동금리,vix
def get_old_data():

    kakao_old = pd.DataFrame(mydb['kakao_all'].find()).drop('_id',axis=1)
    naver_old = pd.DataFrame(mydb['naver_all'].find()).drop('_id',axis=1)
    # unnamed: 0는 날짜가 저장되어있는 칼럼이고, 이 함수가 반환하는 최종 데이터는
    # 날짜(인덱스),  나머지 칼럼
    #      1   2   3   4
    #0606  d   d   3   d
    return kakao_old, naver_old

# 이 함수는 오늘치 데이터를 가져옴. 없는 값은 None으로 반환
# fillna안하는 함수
def daily_update():
    vix = get_vix()

    kakao_score = daily_mongo_update('kakao','kakao_score')
    kakao_else = get_today_data("kakao")

    naver_score = daily_mongo_update('naver','naver_score')
    naver_else = get_today_data("naver")

    kakao = pd.concat([kakao_else,kakao_score],axis=1)
    kakao["변동금리"] = None
    kakao["vix"] = vix

    naver = pd.concat([naver_else,naver_score],axis=1)
    naver["변동금리"] = None
    naver["vix"] = vix

    kakao.columns = ["나스닥","코스피","코스닥","기관합계매수량","기타법인매수량","개인매수량",
                    "외국인합계매수량","종가","등락률","거래량","전일비","score1","score2","score3","변동금리","vix"]
    naver.columns = ["나스닥","코스피","코스닥","기관합계매수량","기타법인매수량","개인매수량",
                    "외국인합계매수량","종가","등락률","거래량","전일비","score1","score2","score3","변동금리","vix"]

    return kakao, naver

# 오늘자 데이터를 fillna해서 빈틈없이만들고
# 기존 데이터에 덧붙힘.
def save_to_mongo():
    a = datetime.now()
    year = f"{a.year}"
    month = f"{a.month}"
    if len(month) < 2:
        month = "0" + month
    day = f"{a.day}"
    if len(day) < 2:
        day = "0" + day
    today_dash = year + "-" + month + "-" + day
    
    # 오늘자 데이터 가져오기, fillna안된 데이터, None 값이 포함된 데이터.
    # 만약 이거를 그냥 db에 넣는다면 여기서 fillna를해야함
    # 그러나 파이썬에서 ffill을 하려면 이전날짜 데이터가 필요하긴 함.
    kakao, naver = daily_update()
    kakao = kakao.reset_index()
    kakao = kakao.rename({'index':'날짜'},axis=1)
    naver = naver.reset_index()
    naver = naver.rename({'index': '날짜'}, axis=1)
    # 기존데이터 가져오기
    # db에저장한다고하면 맨 마지막줄만 불러와도됨.
    kakao_old, naver_old = get_old_data()

    # 기존 데이터에 덧씌우기
    try:
        # 오늘자 컬럼이 존재하면 데이터를 업데이트하지 않음
        kakao_old.loc[today_dash]
        print("오늘 카카오 데이터는 이미 업데이트 되었습니다.")
    except:
        kakao_new = kakao_old.append(kakao)
        kakao_new[["score1","score2","score3"]] = kakao_new[["score1","score2","score3"]].fillna(0)
        kakao_new[["기관합계매수량","기타법인매수량","개인매수량","외국인합계매수량","등락률","거래량","전일비"]] = kakao_new[["기관합계매수량","기타법인매수량","개인매수량","외국인합계매수량","등락률","거래량","전일비"]].fillna(0)
        kakao_new.fillna(method="ffill",inplace=True)
        print(kakao_new.tail(5))
        # 여기를 db에 업데이트하는걸로 바꾸면 됨.
        mydb['kakao_all'].delete_many({})
        mydb['kakao_all'].insert_many(kakao_new.to_dict('records'))

    try:
        naver_old.loc[today_dash]
        print("오늘 네이버 데이터는 이미 업데이트 되었습니다.")
    except:
        naver_new = naver_old.append(naver)
        naver_new[["score1","score2","score3"]] = naver_new[["score1","score2","score3"]].fillna(0)
        naver_new[["기관합계매수량","기타법인매수량","개인매수량","외국인합계매수량","등락률","거래량","전일비"]] = naver_new[["기관합계매수량","기타법인매수량","개인매수량","외국인합계매수량","등락률","거래량","전일비"]].fillna(0)
        naver_new.fillna(method="ffill",inplace=True)
        print(naver_new.tail(5))
        # db에 업데이트하는걸로 바꾸면 됨.
        mydb['naver_all'].delete_many({})
        mydb['naver_all'].insert_many(naver_new.to_dict('records'))




#save_to_mongo()
# print(get_vix())

# print(daily_update())

# 데이터만 가져오려면
#daily_update()

# 스코어만
# kakao_score = daily_mongo_update('kakao','kakao_score')
# print(kakao_score)
# naver_score = daily_mongo_update('naver','naver_score')
# print(naver_score)



# ka_tp = pd.read_csv('/Users/junho/Desktop/streamlit/data/kakao_1_year.csv').rename({'Unnamed: 0':'날짜'},axis=1)
# na_tp = pd.read_csv('/Users/junho/Desktop/streamlit/data/naver_1_year.csv').rename({'Unnamed: 0':'날짜'},axis=1)
# mydb['kakao_all'].insert_many(ka_tp.to_dict('records'))
# mydb['naver_all'].insert_many(na_tp.to_dict('records'))

No Korean fonts found. Please install the font shown belowbecause Hangul can be broken on the chart.
 - NanumBarunGothic font is required.

    1) font install

    !sudo apt-get install -y fonts-nanum
    !sudo fc-cache -fv
    !rm ~/.cache/matplotlib -rf

    2) runtime restart if colab
    
PYKRX works fine regardless of font installation.


No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)
No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


In [2]:
my_client = MongoClient('mongodb://18.181.49.139:27017')
mydb = my_client['final_project']

In [3]:
save_to_mongo()

0
1
score1
score2
score3
0
score1
score2
score3
             날짜       나스닥      코스피     코스닥       기관합계매수량       기타법인매수량  \
376  2022-06-05  12101.42  2670.65  891.51  0.000000e+00  0.000000e+00   
377  2022-06-06  12101.42  2670.65  891.51  0.000000e+00  0.000000e+00   
378  2022-06-07  12101.42  2626.34  873.78 -3.068273e+10  2.108768e+09   
379  2022-06-08  12101.42  2626.15  874.95 -1.372981e+10  1.984313e+08   
0    2022-06-09  12101.42  2625.44  877.18 -1.082364e+10  1.812769e+08   

            개인매수량      외국인합계매수량       종가   등락률        거래량     전일비    score1  \
376  0.000000e+00  0.000000e+00  85700.0  0.00        0.0  2000.0 -0.069880   
377  0.000000e+00  0.000000e+00  85700.0  0.00        0.0  2000.0 -0.846461   
378  1.105126e+11 -8.193864e+10  81900.0 -4.43  2373935.0 -3800.0  0.114813   
379  1.358503e+10 -5.364690e+07  81500.0 -0.49  1317967.0  -400.0  0.018781   
0    1.689807e+10 -6.255708e+09  81300.0 -0.25  1732196.0  -200.0  0.746997   

       score2    score3  변동금리   

In [4]:
my_client = MongoClient('mongodb://18.181.49.139:27017')
mydb = my_client['final_project']
pd.DataFrame(mydb['kakao_all'].find())
pd.DataFrame(mydb['naver_all'].find())

,_id,날짜,나스닥,코스피,코스닥,기관합계매수량,기타법인매수량,개인매수량,외국인합계매수량,종가,등락률,거래량,전일비,score1,score2,score3,변동금리,vix
0,62a1f50fe0326903ee0f1ded,2021-05-25,13657.17,3171.32,962.07,3.717644e+10,-3.254900e+07,-1.805565e+10,-1.908825e+10,358000.0,0.70,433413.0,2500.0,0.205740,0.477670,0.309419,0.24,18.99
1,62a1f50fe0326903ee0f1dee,2021-05-26,13738.00,3168.43,966.06,4.790684e+10,-3.788020e+08,-7.177119e+10,2.424315e+10,363000.0,1.40,543316.0,5000.0,0.618050,0.729894,0.629829,0.24,18.68
2,62a1f50fe0326903ee0f1def,2021-05-27,13736.28,3165.51,974.08,9.866640e+09,9.310600e+07,6.036994e+10,-7.032969e+10,358000.0,-1.38,963669.0,-5000.0,0.050611,0.448271,0.283389,0.24,18.29
3,62a1f50fe0326903ee0f1df0,2021-05-28,13748.74,3188.73,977.46,1.966990e+09,1.733319e+09,1.032580e+10,-1.402611e+10,358000.0,0.00,439112.0,0.0,0.290959,0.542121,0.181454,0.24,17.31
4,62a1f50fe0326903ee0f1df1,2021-05-29,13748.74,3188.73,977.46,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,358000.0,0.00,0.0,0.0,0.748295,0.880630,0.860972,0.24,17.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,62a1f50fe0326903ee0f1f65,2022-06-05,12012.73,2670.65,891.51,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,287500.0,0.00,0.0,-2500.0,0.006367,-0.661489,-0.152210,1.19,17.6
377,62a1f50fe0326903ee0f1f66,2022-06-06,12012.73,2670.65,891.51,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,287500.0,0.00,0.0,-2500.0,-0.562500,-0.714538,-0.703354,1.19,17.6
378,62a1f50fe0326903ee0f1f67,2022-06-07,12012.73,2626.34,873.78,-3.124256e+10,1.374479e+09,3.697569e+10,-7.107612e+09,278000.0,-3.30,660801.0,-9500.0,-0.055032,-0.459467,-0.073785,1.19,18.10
379,62a1f50fe0326903ee0f1f68,2022-06-08,12012.73,2626.15,874.95,1.683491e+09,-2.866465e+08,3.547857e+09,-4.944702e+09,278000.0,0.00,400681.0,0.0,-0.136118,-0.272670,-0.216188,1.19,18.10
